In [ ]:
import yaml
from sqlalchemy import create_engine, text
from sqlalchemy.inspection import inspect

In [ ]:
props = {
    "fact_novelty" : {
        "foreign_key" : {
            "column" : "key_type_novelty",
            "ref_table" : "dim_type_novelty",
            "ref_column" : "key_dim_type_novelty"
        }
    },
}

In [ ]:
def define_primary_key(conn, table_name, column_name):
    try:
        query = text(f'ALTER TABLE {table_name} ADD PRIMARY KEY ({column_name})')
        conn.execute(query)
        conn.commit()
        print(f"Primary key added on {table_name} for column {column_name}.")
    except Exception as e:
        print(f"Error adding primary key on {table_name}: {e}")

def primary_key_exists(inspector, table_name):
    pk_constraint = inspector.get_pk_constraint(table_name)
    return pk_constraint is not None and len(pk_constraint.get('constrained_columns', [])) > 0

def define_foreigns_key(conn, table_name, column_name, ref_table, ref_column):
    try:
        query = text(f'ALTER TABLE {table_name} ADD FOREIGN KEY ({column_name}) REFERENCES {ref_table}({ref_column})')
        conn.execute(query)
        conn.commit()
        print(f"Foreign key added on {table_name} for column {column_name}.")
    except Exception as e:
        print(f"Error adding foreign key on {table_name}: {e}")

def foreign_key_exists(inspector, table_name, column_name):
    fk_constraints = inspector.get_foreign_keys(table_name)
    for fk in fk_constraints:
        if column_name in fk.get('constrained_columns', []):
            return True
    return False

In [ ]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineDestination = create_engine(urlDestination)

inspector = inspect(engineDestination)
tnames = inspector.get_table_names()

with engineDestination.connect() as conn: 
    for table in tnames:
        primary_key_column = f'key_{table}'
        
        columns = inspector.get_columns(table)
        column_names = [column['name'] for column in columns]

        if primary_key_exists(inspector, table):
            print(f"Table {table} already has a primary key. Skipping.")
            continue

        if primary_key_column in column_names:
            define_primary_key(conn, table, primary_key_column)
        else:
            print(f"Column {primary_key_column} does not exist in table {table}.")

    conn.close()

print("\n")

with engineDestination.connect() as conn:
    for table in tnames:
        for column, prop in props.items():
            if column in table:
                for fk, fk_info in prop.items():
                    if not foreign_key_exists(inspector, table, fk_info['column']):
                        define_foreigns_key(conn, table, fk_info['column'], fk_info['ref_table'], fk_info['ref_column'])
                    else:
                        print(f"Foreign key already exists on {table} for column {fk_info['column']}. Skipping.")
                    break
            
    conn.close